In [ ]:
from algorithm import build_dataframe, group_names
from parse_data import get_files, parse_files
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# Initialising the DataFrame
For each calendar (`.ics`) you rename them to the person's name and then put them within `people/`

`get_files` traverses all the files and generates a list of `Calendar` objects which can then be manipulated further, in this case i have `parse_files` converting all the lectures into dictionaries which is then appended to a list.

In [82]:
calendars = get_files()
people = parse_files(calendars)

## Import into Pandas
`build_dataframe` converts a list of dictionaries into a DataFrame itself, e.g. 
```json
[{"name": "Connor", "age": 19}, {"Luce": "age": 19}] -> DataFrame
```
It takes an optional argument drop, defaulting to "OL-Drop-In" to remove all drop-ins as they are available for everyone.


After this, some transformations are applied to group duplicate rows by the Name attribute. 

In [ ]:
df = build_dataframe(people)
df = group_names(df)

An example of the structure of the DataFrame

In [60]:
df.head()

,Day,Start,End,Week Number,Building,Room,Unit Ref,Unit,Type,Group,Lecturer,Names
12,Friday,09:00:00,11:00:00,12,Richmond Building,1.03,M26538,Applied Machine Learning And Data Mining,Work,N/A,"Mohasseb, Alaa",connor azamat jack aden
14,Friday,11:00:00,13:00:00,12,Future Technology Centre,Floor3,M21269,Database Principles,Comp,Gp02,"Nakkas,Haythem",jack aden
13,Friday,15:00:00,17:00:00,12,Future Technology Centre,Floor3,M21269,Database Principles,Comp,Gp04,"Nakkas,Haythem",azamat
